In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp meiosis

In [ ]:
from chewc.core import *
import torch

## meiosis
> Simulating Meisois and Recombination for various Crossing actions

In [ ]:
import torch

def gamma_interference_model(lengths, rate, shape, device):
    """
    Simulate crossover events using a gamma interference model for multiple chromosomes.
    
    Parameters:
    lengths (torch.Tensor): Lengths of each chromosome.
    rate (float): Rate of crossover events.
    shape (float): Shape parameter for the gamma distribution.
    device (torch.device): Device to perform computations on.
    
    Returns:
    List[torch.Tensor]: List of positions of crossover events for each chromosome.
    """
    num_crossovers = torch.poisson(rate * lengths).int()
    crossover_positions = []
    
    for i, length in enumerate(lengths):
        intervals = torch.distributions.gamma.Gamma(shape, rate).sample((num_crossovers[i],)).to(device)
        positions = torch.cumsum(intervals, dim=0)
        positions = positions[positions < length]
        crossover_positions.append(positions)
    
    return crossover_positions

def simulate_gametes(genome: Genome, parent_genomes: torch.Tensor) -> torch.Tensor:
    """
    Simulate the formation of gametes for multiple parents using vectorized operations.

    Args:
        genome (Genome): The Genome instance containing the genetic map and other parameters.
        parent_genomes (torch.Tensor): Genomes of the parents. 
                                       Shape: (num_individuals, ploidy, num_chromosomes, num_loci)

    Returns:
        torch.Tensor: The resultant gametes. 
                      Shape: (num_individuals, ploidy//2, num_chromosomes, num_loci)
    """
    device = genome.device
    genetic_map = genome.genetic_map
    num_individuals, ploidy, num_chromosomes, num_loci = parent_genomes.shape

    chromosome_lengths = genetic_map.max(dim=1).values

    # Simulate crossover positions for all chromosomes at once
    all_crossovers = gamma_interference_model(chromosome_lengths, 1.0, 2.0, device)

    # Initialize gametes tensor
    gametes = torch.zeros_like(parent_genomes[:, :ploidy//2])

    # Apply crossovers to each chromosome
    for chr_idx, crossovers in enumerate(all_crossovers):
        for individual_idx in range(num_individuals):
            for ploid_idx in range(ploidy // 2):
                crossover_mask = torch.zeros(num_loci, device=device, dtype=torch.bool)
                positions_idx = torch.searchsorted(genetic_map[chr_idx], crossovers)
                crossover_mask[positions_idx] = True
                
                # Apply crossover mask to parent genomes to simulate gametes
                gametes[individual_idx, ploid_idx, chr_idx] = torch.where(crossover_mask,
                                                                         parent_genomes[individual_idx, ploid_idx * 2, chr_idx],
                                                                         parent_genomes[individual_idx, ploid_idx * 2 + 1, chr_idx])
    
    return gametes


In [ ]:
ploidy = 2
n_chr = 10
n_loci = 100
n_Ind = 3330
g = Genome(ploidy, n_chr, n_loci)
population = Population()
population.create_random_founder_population(g, n_founders=n_Ind)
init_pop = population.get_genotypes().float()  # gets allele dosage for calculating trait values
simulate_gametes(g,init_pop)[0].shape

Created genetic map


torch.Size([1, 10, 100])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()